In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# %matplotlib inline
plt.style.use("ggplot")

import sklearn
from sklearn.decomposition import TruncatedSVD
import pickle

In [51]:
pip freeze > requiremento.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
amazon_ratings = pd.read_csv('ratings_Beauty.csv')
amazon_ratings = amazon_ratings.dropna()
amazon_ratings.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [5]:
amazon_ratings.shape

(2023070, 4)

In [6]:
# Subset of Amazon Ratings

amazon_ratings1 = amazon_ratings.head(10000)

In [7]:
ratings_utility_matrix = amazon_ratings1.pivot_table(values='Rating', index='UserId', columns='ProductId', fill_value=0)
ratings_utility_matrix.head()

ProductId,0205616461,0558925278,0733001998,0737104473,0762451459,1304139212,1304139220,130414089X,130414643X,1304146537,...,B000052YPE,B000052YPF,B000052YPG,B000052YPH,B000052YPM,B000052YPU,B000052YPV,B000052YPY,B000052YQ0,B000052YQ2
UserId,,,,,,,,,,,,,,,,,,,,,
A00205921JHJK5X9LNP42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A024581134CV80ZBLIZTZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A03056581JJIOL5FSKJY7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A03099101ZRK4K607JVHH,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0505229A7NSH3FRXRR4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


As expected, the utility matrix obtaned above is sparce, I have filled up the unknown values wth 0.

In [8]:
ratings_utility_matrix.shape

(9697, 886)

Transposing the matrix

In [9]:
X = ratings_utility_matrix.T
X.head()

UserId,A00205921JHJK5X9LNP42,A024581134CV80ZBLIZTZ,A03056581JJIOL5FSKJY7,A03099101ZRK4K607JVHH,A0505229A7NSH3FRXRR4,A05492663T95KW63BR75K,A059547920Q3LZVFHLPI3,A07410232KYRFR25CIUGJ,A082796624UNM47DSAI6K,A0864963DOAY7LXGS5I6,...,AZW1HXXYAC15B,AZWRTJPN7NXT,AZWTXHXZXFAYP,AZYQEFB9Y5N22,AZZHB6U54UDYW,AZZHJZP4GQPPZ,AZZNK89PXD006,AZZOFVMQC0BJG,AZZQXL8VDCFTV,AZZTJQ7CQZUD8
ProductId,,,,,,,,,,,,,,,,,,,,,
0205616461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0558925278,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0733001998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0737104473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0762451459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
X.shape

(886, 9697)

Unique products in subset of data

In [11]:
X1 = X

### Decomposing the Matrix

In [12]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(886, 10)

In [25]:
import pickle
print("model trained.saving model..")
filename = './model23.pkl'
pickle.dump(decomposed_matrix, open(filename, 'wb'))
print("model saved")

model trained.saving model..
model saved


In [26]:
import pickle
print("Loading model")
filename = './model23.pkl'
model = pickle.load(open(filename, 'rb'))


Loading model


### Correlation Matrix

In [13]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(886, 886)

correlation_matrix

### Isolating Product ID # 6117036094 from the Correlation Matrix

Assuming the customer buys Product ID # 6117036094 (randomly chosen)

In [14]:
X.index[99]

'6117036094'

Index # of product ID purchased by customer

In [15]:
i = "6117036094"

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

99

Correlation for all items with the item purchased by this customer based on items rated by other customers people who bought the same product

In [16]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(886,)

### Recommending top 10 highly correlated products in sequence

In [17]:
Recommend = list(X.index[correlation_product_ID > 0.90])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:9]

['0205616461',
 '1304146537',
 '1304168522',
 '1304351475',
 '1304482596',
 '1304488608',
 '1304495396',
 '1304511073',
 '1304511081']

In [42]:
l=[]
for i in range(len(X.index)):
    l.append(X.index[i])

In [47]:
product_ID

99

In [65]:
correlation_product_ID


array([-0.06221456, -0.03299528,  0.29644143, -0.54787359,  0.23304435,
       -0.54192848, -0.17012112,  0.25586559, -0.54182421,  0.19019763,
       -0.17012113,  0.18936782, -0.14691446, -0.54627476, -0.54192848,
       -0.17012113, -0.17012113, -0.26236833, -0.14690788,  0.29722992,
        0.18213902, -0.18808876, -0.08834396,  0.18921724,  0.1828102 ,
        0.33447614,  0.33864052, -0.36758108,  0.18921724,  0.32478782,
        0.32128005, -0.54202246,  0.07739998,  0.33423809,  0.18934072,
       -0.54192848,  0.33423809, -0.06704846, -0.54171446, -0.3984565 ,
       -0.39742789,  0.33447614, -0.54246897, -0.54192848, -0.54192848,
       -0.54192848,  0.18921724, -0.54192848, -0.54184705, -0.3984565 ,
       -0.44099195, -0.17229448, -0.03658046, -0.33290086,  0.08021295,
       -0.27982783,  0.6756894 , -0.09887786,  0.82588012,  0.38284362,
        0.35493941, -0.12869522,  0.13075313,  0.15885668,  0.21834559,
        0.86864307,  0.24935297, -0.66232915,  0.47397361,  0.60

In [74]:

correlation_matrix = np.corrcoef(model)

i = "6117036094"

product_names = list(l)
product_ID = product_names.index(i)


correlation_product_ID = correlation_matrix[product_ID]

#Recommend = list(l(str([correlation_product_ID > 0.90])))
Recommend=[]
for k in range(len(l)):
    if(correlation_product_ID[k] > 0.90):
        Recommend.append(l[k])

Recommend.remove(i) 

Recommend[0:9]

['6162751473',
 '9788075509',
 '9790785682',
 'B000027S8A',
 '6162751473',
 '9788075509',
 '9790785682',
 'B000027S8A',
 '6162751473']